# Always Required

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
# import copy
# import os
# from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pygama.flow import DataLoader
from pygama.flow import FileDB
from lgdo import LH5Store
# from pygama.math import histogram as pgh
# from pygama.math import peak_fitting as pf
from dspeed import processors

In [ ]:
mpl.rcParams["font.family"] = "serif"

## FileDB and DataLoader Setup

### From Scratch

In [ ]:
fdb = FileDB(config="../processing/metadata/dataloader_configs/cage_filedb_config.json", scan=True)
fdb.scan_tables_columns()
fdb.to_disk("../processing/cage_filedb.lh5", "o")

In [ ]:
dl = DataLoader(config="../processing/metadata/dataloader_configs/cage_loader_config.json",
                filedb=fdb)

### Load Existing

In [ ]:
fdb = FileDB("../processing/cage_filedb.lh5")

In [ ]:
# Run this if FDB needs to be updated and then re-saved to disk
og_col = fdb.columns
fdb.scan_files()
fdb.set_file_status()
fdb.set_file_sizes()
fdb.scan_tables_columns()
assert fdb.columns == og_col
fdb.to_disk("../processing/cage_filedb.lh5", "o")

In [ ]:
dl = DataLoader(config="../processing/metadata/dataloader_configs/cage_loader_config.json",
                filedb=fdb)

# Process Data

I'm going to want to load waveforms and compute a parameter based on that. 
Ideas for EQ parameter:

1. Just timepoint data, compare to Joule's tp20-tp2 parameter
    1. Load DSP parameters
2. Integral underneath waveform up to tp20 (or other tps)
    1. Need tp from DSP as well as waveform
3. Something like A/E but only with the waveform up to tp50 
    1. Need tp from DSP as well as waveform

In [ ]:
dsp_config_dir = "../processing/metadata/dsp"
config_files = glob.glob(f"{dsp_config_dir}/dsp_cyc*.json")
parser = parse.compile(dsp_config_dir + "/dsp_cyc{}.json")
config_breakpoints = []
for file in config_files:
    config_breakpoints.append(int(parser.parse(file)[0]))
config_breakpoints = np.array(sorted(config_breakpoints))

In [ ]:
def set_dsp_config(cycle):
    if cycle < config_breakpoints[0]:
        print("DSP configuration parameters not recorded for this cycle, bailing...")
        return
    
    if cycle < config_breakpoints[-1]:
        config_cyc = config_breakpoints[ np.where(config_breakpoints > cycle)[0][0] - 1 ]
    else:
        print("Caution: Using latest DSP configuration, which may not be up to date")
        config_cyc = config_breakpoints[-1]
    
    config_dict = json.load(f" {dsp_config_dir}/dsp_cyc{config_cyc}.json ")dsp_db
    dsp_db["40K"]

In [ ]:
cage_lh5 = os.environ.get('CAGE_LH5')
def init_dspdb():
    dsp_db = {
        "40K": {
            "tp": {
                "0": 0,
                "2": 0,
                "20": 0,
                "50": 0

            },
            "energy": {
                "trapEmax": 0,
                "trapEftp": 0

            }
        }
    }
    return dsp_db

def set_dspdb(cycle):
    dsp_db = init_dspdb()
    dsp_file = glob.glob(f"{cage_lh5}/dsp/cage_run*_cyc{cycle}_dsp.lh5")[0]
    print(dsp_file)

In [ ]:

dsp_config = {
  "outputs": [
    "channel", "timestamp", "daqenergy",
    "bl", "bl_sig", "bl_slope_ftp", "bl_sig", "bl_slope", "bl_intercept",
    "wf_max", "wf_argmax", 
    "trapEmax", "tp_0", "trapEftp", "trapEmax_ctc"
    "A_10", "AoE", "triE", "ToE",
    "tp_max", "tp_02", "tp_05", "tp_10", "tp_20", "tp_30", "tp_40", "tp_50",
    "tp_60", "tp_70", "tp_80", "tp_90", "tp_96"
  ],
  "processors":{
    "bl, bl_sig, bl_slope, bl_intercept":{
      "function": "linear_slope_fit",
      "module": "pygama.dsp.processors",
      "args" : ["waveform[0: 3500]", "bl","bl_sig", "bl_slope","bl_intercept"],
      "unit": ["ADC","ADC","ADC","ADC"]
    },
    "wf_blsub":{
      "function": "subtract",
      "module": "numpy",
      "args": ["waveform", "bl", "wf_blsub"],
      "prereqs": ["waveform", "bl"],
      "unit": "ADC"
    },
    "tp_min, tp_max, wf_min, wf_max":{
      "function": "min_max",
      "module": "pygama.dsp.processors",
      "args": ["wf_blsub", "tp_min", "tp_max", "wf_min", "wf_max"],
      "unit": ["ns","ns","ADC", "ADC"],
      "prereqs": ["wf_blsub"]
    },
    
  }
}

# Testing Processors

In [ ]:
cyc = 4259
dl.set_files(f"cycle == {cyc}")
dl.set_output(columns = ["waveform", "tp_0", "tp_02", "tp_05", "tp_10", "tp_20", "tp_50", "tp_max", "trapEmax", "dcr", "ToE", "AoE"], fmt="pd.DataFrame")
# dl.set_output(columns = ["tp_0", "tp_02", "tp_20", "tp_50", "tp_max", "trapEmax", "dcr", "ToE", "AoE"], fmt="pd.DataFrame")
d = dl.load()

In [ ]:
def bl_subtract(row):
    bl_avg = np.average(row.waveform_values[:500])
    
    return row.waveform_values - bl_avg

In [ ]:
d['wf_blsub'] = d.apply(bl_subtract, axis=1)

## Conventional

In [ ]:
d['tp20-0'] = d.apply(lambda r: r.tp_20 - r.tp_0, axis=1)
d['tp20-2'] = d.apply(lambda r: r.tp_20 - r.tp_02, axis=1)
d['tp20-5'] = d.apply(lambda r: r.tp_20 - r.tp_05, axis=1)
d['tp20-10'] = d.apply(lambda r: r.tp_20 - r.tp_10, axis=1)
d['tp50-0'] = d.apply(lambda r: r.tp_50 - r.tp_0, axis=1)
d['tp50-2'] = d.apply(lambda r: r.tp_50 - r.tp_02, axis=1)
d['tp50-5'] = d.apply(lambda r: r.tp_50 - r.tp_05, axis=1)
d['tp50-10'] = d.apply(lambda r: r.tp_50 - r.tp_10, axis=1)
d['tp50-20'] = d.apply(lambda r: r.tp_50 - r.tp_20, axis=1)


In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("dcr")
plt.title("DCR")
plt.hist2d(d['trapEmax'], d['dcr'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 200, 200)), norm='log')
plt.savefig(f"plots/EQ/cyc{cyc}_dcr.png")

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("T/E")
plt.title("T/E")
plt.hist2d(d['trapEmax'], d['ToE'], bins=(np.linspace(0, 10000, 500), np.linspace(-.1, 1, 100)), norm='log')
plt.savefig(f"plots/EQ/cyc{cyc}_ToE.png")

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp20 - tp0")
plt.title("tp20-tp0")
plt.hist2d(d['trapEmax'], d['tp20-0'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 2000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp50 - tp0")
plt.title("tp50-tp0")
plt.hist2d(d['trapEmax'], d['tp50-0'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 2000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp20 - tp02")
plt.title("Joule's tp20-tp02")
plt.hist2d(d['trapEmax'], d['tp20-2'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp50 - tp02")
plt.title("tp50-tp02")
plt.hist2d(d['trapEmax'], d['tp50-2'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp20 - tp05")
plt.title("tp20-tp05")
plt.hist2d(d['trapEmax'], d['tp20-5'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp50 - tp05")
plt.title("tp50-tp05")
plt.hist2d(d['trapEmax'], d['tp50-5'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp20 - tp10")
plt.title("tp20-tp10")
plt.hist2d(d['trapEmax'], d['tp20-10'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp50 - tp05")
plt.title("tp50-tp10")
plt.hist2d(d['trapEmax'], d['tp50-10'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 1000, 100)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("tp50 - tp20")
plt.title("tp50-tp20")
plt.hist2d(d['trapEmax'], d['tp50-20'], bins=(np.linspace(0, 10000, 500), np.linspace(-50, 2000, 100)), norm='log')

## Integration

Improvement ideas: 
- Waveform smoothing

In [ ]:
def smooth_wf(row, length=5):
    smoothed = np.zeros_like(row.wf_blsub)
    processors.moving_window_left(row.wf_blsub, length, smoothed)
    return smoothed

In [ ]:
full_d['wf_blsub_smoothed'] = full_d.apply(smooth_wf, axis=1)

In [ ]:
def integrate_tp_smoothed(row, tp_start, tp_end):
    try:
        ind_start = int( row[tp_start] / row.waveform_dt )
        ind_end = int( row[tp_end] / row.waveform_dt )
    except ValueError:
        return np.nan
    return np.sum(row.wf_blsub_smoothed[ind_start:ind_end])

def integrate_tp(row, tp_start, tp_end):
    try:
        ind_start = int( row[tp_start] / row.waveform_dt )
        ind_end = int( row[tp_end] / row.waveform_dt )
    except ValueError:
        return np.nan
    return np.sum(row.wf_blsub[ind_start:ind_end])

In [ ]:
def calc_plot_int(tp_start, tp_end, df, int_bins=np.linspace(-100, 500, 400), save=False):
    start = tp_start[3:]
    end = tp_end[3:]
    int_name = f'int_{start}_{end}'
    df[int_name] = df.apply(integrate_tp, args=(tp_start, tp_end), axis=1)
    
    plt.figure()
    plt.title(f"Integration {tp_start} to {tp_end}")
    plt.xlabel("trapEmax")
    plt.ylabel(f"int {tp_start} - {tp_end}")
    plt.hist2d(df['trapEmax'], df[int_name]/df['trapEmax'], bins=(np.linspace(0, 10000, 500), int_bins), norm='log')
    
    if save:
        plt.savefig(f"plots/EQ/cyc{cyc}_int_{tp_start}_{tp_end}.png")

In [ ]:
calc_plot_int("tp_0", "tp_20", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_0", "tp_50", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_02", "tp_20", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_02", "tp_50", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_05", "tp_20", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_05", "tp_50", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_10", "tp_20", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_10", "tp_50", d, np.linspace(-5, 20, 100), True)

In [ ]:
calc_plot_int("tp_20", "tp_50", d, np.linspace(-5, 20, 100), True)

## Derivative

In [ ]:
def early_curr_max(row, length=5, tp_end='tp_50'):
    if np.isnan(row[tp_end]):
        return np.nan
    ind = int( row[tp_end]/row.waveform_dt )
    curr_wf = np.zeros( ind - length)
    processors.avg_current(row.wf_blsub[:ind], length, curr_wf)
    time_before_max = row.tp_max - int( np.argmax(curr_wf) ) * row.waveform_dt
    
    return np.max(curr_wf), time_before_max

In [ ]:
full_d[["A_max_50", "A_argmax_50"]] = full_d.apply(early_curr_max, axis=1, result_type='expand', args=(5, 'tp_50'))

In [ ]:
full_d[["A_max_20", "A_argmax_20"]] = full_d.apply(early_curr_max, axis=1, result_type='expand', args=(5, 'tp_20'))

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("AoE")
plt.title("AoE up to tp50")
plt.hist2d(full_d['trapEmax'], full_d['A_max_50']/full_d['trapEmax'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.1, 1, 200)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("AoE")
plt.title("AoE up to tp20")
plt.hist2d(full_d['trapEmax'], full_d['A_max_20']/full_d['trapEmax'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.1, 1, 200)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("A_argmax - tp0")
plt.title("Time from max of derivative to tp_max")
plt.hist2d(d['trapEmax'], d['A_argmax_50']/d['trapEmax'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.1, 100, 200)), norm='log')

In [ ]:
plt.xlabel("trapEmax")
plt.ylabel("A_argmax - tp0")
plt.title("Time from max of derivative to tp_max")
plt.hist2d(d['trapEmax'], d['A_argmax_20']/d['trapEmax'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.1, 100, 200)), norm='log')

## Double Derivative?

This feels very promising - main thing is to figure out how we want to calculate the derivative
- AoE method: using avg_current from dsp
    - Need to optimize length parameter
- T/E method: using triangle filter
    - Need to optimize rise/fall times
    
Optimization based on width of double_curr distribution at 1460/2615

### avg_curr

In [ ]:
def double_derv(row, length=5):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub) - length)
    doublecurr_wf = np.zeros( len(wf_blsub) - 2*length )
    processors.avg_current(wf_blsub, length, curr_wf)
    processors.avg_current(curr_wf, length, doublecurr_wf)
    return np.max(doublecurr_wf)

def double_derv_wf(row, length=5):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub) - length)
    doublecurr_wf = np.zeros( len(wf_blsub) - 2*length )
    processors.avg_current(wf_blsub, length, curr_wf)
    processors.avg_current(curr_wf, length, doublecurr_wf)
    return doublecurr_wf

In [ ]:
d['double_curr'] = d.apply(double_derv, args=(2,), axis=1)
plt.figure()
plt.title(f"Double Derivative Length = 2")
plt.xlabel("trapEmax")
plt.ylabel(f"double_curr")
plt.hist2d(d['trapEmax'], d['double_curr'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 100, 100)), norm='log')
plt.savefig(f"plots/EQ/cyc{cyc}_doublederv.png")

In [ ]:
d['double_curr'] = d.apply(double_derv, args=(10,), axis=1)
plt.figure()
plt.title(f"Double Derivative Length = 10")
plt.xlabel("trapEmax")
plt.ylabel(f"double_curr")
plt.hist2d(d['trapEmax'], d['double_curr'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 200, 100)), norm='log')

In [ ]:
d['double_curr'] = d.apply(double_derv, args=(1,), axis=1)
plt.figure()
plt.title(f"Double Derivative Length = 1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_curr")
plt.hist2d(d['trapEmax'], d['double_curr'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 200, 100)), norm='log')

In [ ]:
d['double_curr'] = d.apply(double_derv, args=(3,), axis=1)
plt.figure()
plt.title(f"Double Derivative Length = 3")
plt.xlabel("trapEmax")
plt.ylabel(f"double_curr")
plt.hist2d(d['trapEmax'], d['double_curr'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 200, 100)), norm='log')

In [ ]:
for i in range(10,20):
    dcurr_wf = double_derv_wf(d.iloc[i], 2)
    plt.plot(dcurr_wf[:-1])
    plt.xlim(3800, 4100)

In [ ]:
for i in range(10,20):
    dcurr_wf = double_derv_wf(d.iloc[i], 2)
    plt.plot( [np.max(dcurr_wf[:-1])]*10 )

In [ ]:
def plot_aligned_wf(df, length=1000, **kwargs):
    half = int( length/2 )
    norm_wfs = df.wf_blsub / np.array([max(x) for x in df.wf_blsub])
    aligned_wfs = []
    for i, row in df.iterrows():
        if not np.isnan(row.tp_50):
            aligned_wfs.append( norm_wfs.loc[i][int(row.tp_50/row.waveform_dt)-half : int(row.tp_50/row.waveform_dt)+half] )

    for wf in aligned_wfs:
        plt.plot(wf, **kwargs)

In [ ]:
plot_aligned_wf(.query("double_curr > 35 and double_curr < 50").iloc[:20], 1000, color='blue', linewidth=0.1)
plot_aligned_wf(d.query("double_curr < 35").iloc[:20], length=1000, color='red', linewidth=0.1)

### Triangle Filter

In [ ]:
def double_tri(row, rise=2, fall=2):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub))
    doublecurr_wf = np.zeros( len(wf_blsub))
    processors.asym_trap_filter(wf_blsub, rise, 0, fall, curr_wf)
    processors.asym_trap_filter(curr_wf, rise, 0, fall, doublecurr_wf)
    return np.max(doublecurr_wf)

def double_tri_wf(row, rise=2, fall=2):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub))
    doublecurr_wf = np.zeros( len(wf_blsub))
    processors.asym_trap_filter(wf_blsub, rise, 0, fall, curr_wf)
    processors.asym_trap_filter(curr_wf, rise, 0, fall, doublecurr_wf)
    return doublecurr_wf

In [ ]:
d['double_tri'] = d.apply(double_tri, axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 2, Fall=2")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(5, 1), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 5, Fall=1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(10, 1), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 10, Fall=1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(10, 5), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 10, Fall=5")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(5, 2), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 5, Fall=2")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(2, 1), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 2, Fall=1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
plt.figure()
plt.title(f"Double Triangle Rise = 2, Fall=1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')
plt.axhline(80)
plt.axhline(130)

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(4, 2), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 4, Fall=2")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(3, 2), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 3, Fall=2")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(3, 1), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 3, Fall=1")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
d['double_tri'] = d.apply(double_tri, args=(2, 3), axis=1)
plt.figure()
plt.title(f"Double Triangle Rise = 2, Fall=3")
plt.xlabel("trapEmax")
plt.ylabel(f"double_tri")
plt.hist2d(d['trapEmax'], d['double_tri'], bins=(np.linspace(0, 10000, 500), np.linspace(0, 500, 100)), norm='log')

In [ ]:
for i in range(10,20):
    dcurr_wf = double_tri_wf(d.iloc[i], 2, 1)
    plt.plot(dcurr_wf[:-1])
    plt.xlim(3800, 4100)

In [ ]:
for i in range(10,20):
    dcurr_wf = double_tri_wf(d.iloc[i], 2, 1)
    plt.plot( [np.max(dcurr_wf[:-1])]*10 )

In [ ]:
plot_aligned_wf(d.query("double_tri > 80 and double_curr < 130").iloc[:20], 1000, color='blue', linewidth=0.1)
plot_aligned_wf(d.query("double_tri < 80").iloc[:20], length=1000, color='red', linewidth=0.1)

# Double Derivative Optimization

At some point I might test using different parameters for the first and second derivatives, but I'm not convinced that's super useful

In [ ]:
def gaus(x, mu, sig, A):
    return A*np.exp(-(x- mu)**2/sig**2)

def percentile_mean_std(lo_per, hi_per, data, show=False):
    lo, hi = np.percentile(data, [lo_per, hi_per])
    mask = (data > lo) & (data < hi)
    if show:
        print(lo, hi)
    return np.average(data[mask]), np.std(data[mask])

In [ ]:
cycle = 4292

In [ ]:
dl.reset()
dl.set_files(f"cycle == {cycle}")
dl.set_output(columns = ["waveform", "trapEmax"], fmt="pd.DataFrame")
el = dl.build_entry_list()
d = dl.load(el[:500])

In [ ]:
plt.hist(d.trapEmax, bins=np.linspace(0, 6000, 4000))
plt.yscale('log')
plt.xlim(3000, 4000)

In [ ]:
elo = 3100
ehi = 3200

dl.reset()
dl.set_files(f"cycle == {cycle}")
dl.set_output(columns = ["waveform", "tp_0", "tp_02", "tp_20", "tp_50", "tp_max", "trapEmax", "dcr", "ToE", "AoE"], fmt="pd.DataFrame")
dl.set_cuts({"hit": f"trapEmax > {elo} and trapEmax < {ehi}"})
d = dl.load()

In [ ]:
dl.reset()
dl.set_files(f"cycle == {cycle}")
dl.set_output(columns = ["waveform", "tp_0", "tp_02", "tp_20", "tp_50", "tp_max", "trapEmax", "dcr", "ToE", "AoE"], fmt="pd.DataFrame")
# dl.set_cuts({"hit": f"trapEmax > {elo} and trapEmax < {ehi}"})
full_d = dl.load()

In [ ]:
def bl_subtract(row):
    bl_avg = np.average(row.waveform_values[:500])
    return list(row.waveform_values - bl_avg)

In [ ]:
full_d['wf_blsub'] = full_d.apply(bl_subtract, axis=1)

## avg_curr

In [ ]:
def double_derv(row, length=5):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub) - length)
    doublecurr_wf = np.zeros( len(wf_blsub) - 2*length )
    processors.avg_current(wf_blsub, length, curr_wf)
    processors.avg_current(curr_wf, length, doublecurr_wf)
    return np.max(doublecurr_wf)

In [ ]:
for l in range(1, 8):
    double_curr = full_d.apply(double_derv, args=(l,), axis=1)
    plt.figure()
    plt.hist2d(full_d.trapEmax, double_curr, bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 100, 100)), norm='log' )
    plt.title(f"length = {l}")
    # coeff = pf.gauss_mode_width_max(hist, bins, n_bins=11)[0]
    # plt.plot(bins, gaus(bins, *coeff))
    # plt.yscale('log')
    print(l, np.std(double_curr)/np.mean(double_curr))
    # print(l, np.ptp(double_curr)/np.mean(double_curr))
    # print(l, coeff[1])

In [ ]:
best_length = 2

In [ ]:
dl.reset()
dl.set_files(f"cycle == {cycle}")
dl.set_output(columns = ["waveform", "trapEmax"], fmt="pd.DataFrame")
full_d = dl.load()

In [ ]:
full_d['wf_blsub'] = full_d.apply(bl_subtract, axis=1)

In [ ]:
full_d['double_curr'] = full_d.apply(double_derv, args=(best_length,), axis=1)

In [ ]:
plt.figure()
plt.title(f"Double Derivative Length = {best_length}")
plt.xlabel("trapEmax")
plt.ylabel(f"double_curr")
plt.hist2d(full_d['trapEmax'], full_d['double_curr'], bins=(np.linspace(0, 10000, 500), np.linspace(-0.01, 100, 100)), norm='log')
plt.axhline(18)

In [ ]:
plot_aligned_wf(full_d.query("trapEmax > 100 and double_curr > 18").iloc[:20], 250, color='blue', linewidth=0.2)
plot_aligned_wf(full_d.query("trapEmax > 100 and double_curr < 18").iloc[:20], length=250, color='red', linewidth=0.2)

## Triangle Filter

In [ ]:
def double_tri(row, rise=2, fall=2):
    wf_blsub = row.wf_blsub
    curr_wf = np.zeros( len(wf_blsub))
    doublecurr_wf = np.zeros( len(wf_blsub))
    processors.asym_trap_filter(wf_blsub, rise, 0, fall, curr_wf)
    processors.asym_trap_filter(curr_wf, rise, 0, fall, doublecurr_wf)
    return np.max(doublecurr_wf)

In [ ]:
double_curr = d.apply(double_tri, args=(2, 5), axis=1)
plt.figure()
hist, bins, _ = plt.hist(double_curr, bins=40)
plt.title(f"({2},{5})")
coeff = percentile_mean_std(5, 95, double_curr, True)
print(coeff)
# plt.plot(bins, gaus(bins, *coeff))

In [ ]:
for rise in range(2, 6):
    for fall in range(2, 6):
        double_curr = full_d.apply(double_tri, args=(rise, fall), axis=1)
        plt.figure()
        plt.hist2d(full_d.trapEmax, double_curr, bins=(np.linspace(0, 6000, 4000), np.linspace(0, 100, 100)), norm='log' )
        print(rise, fall, np.std(double_curr)/np.mean(double_curr))
        # plt.title(f"({rise},{fall})")
        # coeff = pf.gauss_mode_width_max(hist, bins, n_bins=11)[0]
        # plt.plot(bins, gaus(bins, *coeff))
        # plt.yscale('log')
        # coeff = percentile_mean_std(5, 95, double_curr)
        # print(rise, fall, coeff, coeff[1]/coeff[0])